In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import branca, geopandas, folium

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/biketheftdataset/Bicycle_Thefts_Toronto_geo.csv
/kaggle/input/biketheftdataset/Bicycle_Theft_Ottawa_geo.csv
/kaggle/input/geo-dataset/Bicycle_Theft_Ottawa.csv
/kaggle/input/geo-dataset/pop_toronto.geojson
/kaggle/input/geo-dataset/pop_ottawa.geojson
/kaggle/input/geo-dataset/Bicycle_Thefts_Toronto.csv
/kaggle/input/bike-dataset/pop_bikes_toronto.csv
/kaggle/input/bike-dataset/pop_toronto.csv
/kaggle/input/bike-dataset/pop_bikes_ottawa.csv
/kaggle/input/bike-dataset/pop_ottawa.csv


In [6]:
#import datasets
dp_ottawa=pd.read_csv("../input/bike-dataset/pop_ottawa.csv",header=0,sep=',')
dp_toronto=pd.read_csv("../input/bike-dataset/pop_toronto.csv",header=0)
dbikes_ottawa=pd.read_csv("../input/bike-dataset/pop_bikes_ottawa.csv",header=0,sep=',')
dbikes_toronto=pd.read_csv("../input/bike-dataset/pop_bikes_toronto.csv",header=0)


#general info
print(dp_ottawa.shape)
print(dp_toronto.shape)
print(dbikes_ottawa.shape)
print(dbikes_toronto.shape)

# print(dp_ottawa.columns.tolist())
# print(dbikes_ottawa.columns.tolist())
# dbikes_ottawa['GeoUID']

(204, 11)
(572, 11)
(204, 10)
(572, 10)


In [7]:
Data_Ottawa_pop = dp_ottawa.merge(dbikes_ottawa, how = 'inner', left_on='GeoUID', right_on='GeoUID') #Joint data of population and bikes for ottawa
Data_Toronto_pop = dp_toronto.merge(dbikes_toronto, how = 'inner', left_on='GeoUID', right_on='GeoUID') #Joint data of population and bikes for ottawa

Data_Ottawa_pop=Data_Ottawa_pop.rename(columns={'v_CA16_401: Population, 2016':'PopulationCT','v_CA16_406: Population density per square kilometre':'PopulationDensity',
                                               'v_CA16_407: Land area in square kilometres':'LandArea(skm)','v_CA16_5809: Bicycle':'BicycleF','v_CA16_5808: Bicycle':'BicycleM','v_CA16_5807: Bicycle':'BicycleT'})

Data_Toronto_pop=Data_Toronto_pop.rename(columns={'v_CA16_401: Population, 2016':'PopulationCT','v_CA16_406: Population density per square kilometre':'PopulationDensity',
                                               'v_CA16_407: Land area in square kilometres':'LandArea(skm)','v_CA16_5809: Bicycle':'BicycleF','v_CA16_5808: Bicycle':'BicycleM','v_CA16_5807: Bicycle':'BicycleT'})

Data_Ottawa_pop.loc[132,'BicycleT']=0
Data_Ottawa_pop['BicycleT']=pd.to_numeric(Data_Ottawa_pop['BicycleT'])
Data_Toronto_pop['BicycleT'] = pd.to_numeric(Data_Toronto_pop['BicycleT'])

Data_Ottawa_pop.shape, Data_Toronto_pop.shape

((204, 20), (572, 20))

In [8]:
#importing theft info
data_ottawa=pd.read_csv('../input/biketheftdataset/Bicycle_Theft_Ottawa_geo.csv',header=0)
data_toronto=pd.read_csv('../input/biketheftdataset/Bicycle_Thefts_Toronto_geo.csv',header=0)

print(data_ottawa.columns.tolist())
print(data_toronto.columns.tolist())

['ID', 'Division', 'Sector', 'Report_Date', 'Incident_Date', 'Incident_Weekday', 'Location_Type', 'Primary_Offence', 'Neighbourhood', 'Bicycle_Status', 'Bicycle_Value', 'Bicycle_Make', 'Bicycle_Model', 'Bicycle_Type', 'Bicycle_Frame', 'Bicycle_Colour', 'Bicycle_Speed', 'x', 'y', 'ObjectId', 'index_right', 'GeoUID']
['X', 'Y', 'FID', 'Index_', 'event_unique_id', 'Primary_Offence', 'Occurrence_Date', 'Occurrence_Year', 'Occurrence_Month', 'Occurrence_Day', 'Occurrence_Time', 'Division', 'City', 'Location_Type', 'Premise_Type', 'Bike_Make', 'Bike_Model', 'Bike_Type', 'Bike_Speed', 'Bike_Colour', 'Cost_of_Bike', 'Status', 'Hood_ID', 'Neighbourhood', 'Lat', 'Long', 'index_right', 'GeoUID']


In [9]:
Toronto_data=data_toronto[['Index_', 'event_unique_id', 'Primary_Offence',
       'Occurrence_Date', 'Occurrence_Year', 'Occurrence_Month',
       'Occurrence_Day', 'Occurrence_Time', 'Division', 'City',
       'Location_Type', 'Premise_Type', 'Bike_Make', 'Bike_Model', 'Bike_Type',
       'Bike_Speed', 'Bike_Colour', 'Cost_of_Bike', 'Status', 'Hood_ID',
       'Neighbourhood', 'Lat', 'Long', 'GeoUID']].copy()
Ottawa_data=data_ottawa[['ID', 'Division', 'Sector', 'Report_Date', 'Incident_Date',
       'Incident_Weekday', 'Location_Type', 'Primary_Offence', 'Neighbourhood',
       'Bicycle_Status', 'Bicycle_Value', 'Bicycle_Make', 'Bicycle_Model',
       'Bicycle_Type', 'Bicycle_Frame', 'Bicycle_Colour', 'Bicycle_Speed', 'x',
       'y','GeoUID']].copy()

#configuring occurrence date and day
day_type=pd.api.types.CategoricalDtype(categories=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], ordered=True)

#converting to daytime64 datatype
Toronto_data['Occurrence_Date']=pd.to_datetime(Toronto_data['Occurrence_Date']) 
#referring the datime to the day of the week
Toronto_data['Week_day']=Toronto_data['Occurrence_Date'].dt.day_name() 
#converting that dataype to the custom dataype
Toronto_data["Week_day"] = Toronto_data["Week_day"].astype(day_type)

Ottawa_data['Incident_Date']=pd.to_datetime(Ottawa_data['Incident_Date']) 
Ottawa_data['Week_day']=Ottawa_data['Incident_Date'].dt.day_name() 
Ottawa_data["Week_day"] = Ottawa_data["Week_day"].astype(day_type)

Ottawa_data.info(), Toronto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5652 entries, 0 to 5651
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                5652 non-null   int64         
 1   Division          5652 non-null   object        
 2   Sector            5652 non-null   int64         
 3   Report_Date       5652 non-null   object        
 4   Incident_Date     5652 non-null   datetime64[ns]
 5   Incident_Weekday  5652 non-null   int64         
 6   Location_Type     5652 non-null   object        
 7   Primary_Offence   5652 non-null   object        
 8   Neighbourhood     5652 non-null   object        
 9   Bicycle_Status    5652 non-null   object        
 10  Bicycle_Value     5019 non-null   float64       
 11  Bicycle_Make      4890 non-null   object        
 12  Bicycle_Model     3367 non-null   object        
 13  Bicycle_Type      5381 non-null   object        
 14  Bicycle_Frame     5337 n

(None, None)

In [10]:
JointOttawa=pd.merge(Data_Ottawa_pop, Ottawa_data, on='GeoUID', how='right')
JointToronto=pd.merge(Data_Toronto_pop, Toronto_data, on='GeoUID', how='right')



JointToronto.drop(['v_CA16_1: Age Stats', 'Type_y', 'Region Name_y',
       'Area (sq km)_y', 'Population _y', 'Dwellings _y', 'Households _y','Index_', 'event_unique_id','City','Premise_Type','Hood_ID'],axis=1,inplace=True)
JointOttawa.drop(['v_CA16_1: Age Stats', 'Type_y', 'Region Name_y',
       'Area (sq km)_y', 'Population _y', 'Dwellings _y', 'Households _y','ID','Sector','Report_Date','Incident_Weekday','Bicycle_Frame'],axis=1,inplace=True)


In [11]:

JointOttawa=JointOttawa.rename(columns={'Type_x':'Type', 'Region Name_x':'RegionName', 'Area (sq km)_x':'Area(sqKm)', 'Population _x':'Population',
       'Dwellings _x':'Dwellings', 'Households _x':'Households','Bicycle_Status':'Status','x':'Lat','y':'Long','Bicycle_Value':'Cost_of_Bike','Incident_Date':'Occurrence_Date',
       'Bicycle_Make':'Bike_Make', 'Bicycle_Model':'Bike_Model','Bicycle_Type':'Bike_Type', 'Bicycle_Colour':'Bike_Colour', 'Bicycle_Speed':'Bike_Speed'})

JointToronto=JointToronto.rename(columns={'Type_x':'Type', 'Region Name_x':'RegionName', 'Area (sq km)_x':'Area(sqKm)', 'Population _x':'Population',
       'Dwellings _x':'Dwellings', 'Households _x':'Households'})


#formatting date time 
JointOttawa['Occurrence_Year']=JointOttawa['Occurrence_Date'].dt.year
JointOttawa['Occurrence_Month']=JointOttawa['Occurrence_Date'].dt.month
JointOttawa['Occurrence_Day']=JointOttawa['Occurrence_Date'].dt.day
JointOttawa['Occurrence_Time'] = pd.to_datetime(JointOttawa['Occurrence_Date'],format= '%H:%M:%S' ).dt.time
JointToronto['BikeDensity']=JointToronto['BicycleT']/JointToronto['LandArea(skm)'] #Numer of bikes by Km^2 Toronto
JointOttawa['BikeDensity']=JointOttawa['BicycleT']/JointOttawa['LandArea(skm)']  #Numer of bikes by Km^2 Toronto

JointToronto['City']='Toronto'
JointOttawa['City']='Ottawa'

JointToronto.sort_index(axis=1,inplace=True)
JointOttawa.sort_index(axis=1,inplace=True)

JointOttawa.info(), JointToronto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5652 entries, 0 to 5651
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Area(sqKm)         5652 non-null   float64       
 1   BicycleF           5652 non-null   object        
 2   BicycleM           5652 non-null   object        
 3   BicycleT           5652 non-null   int64         
 4   BikeDensity        5652 non-null   float64       
 5   Bike_Colour        5262 non-null   object        
 6   Bike_Make          4890 non-null   object        
 7   Bike_Model         3367 non-null   object        
 8   Bike_Speed         5652 non-null   int64         
 9   Bike_Type          5381 non-null   object        
 10  City               5652 non-null   object        
 11  Cost_of_Bike       5019 non-null   float64       
 12  Division           5652 non-null   object        
 13  Dwellings          5652 non-null   int64         
 14  GeoUID  

(None, None)

In [12]:
CityJoint=pd.concat([JointOttawa, JointToronto],ignore_index=True)
CityJoint[['BicycleF', 'BicycleM', 'BicycleT']] = CityJoint[['BicycleF', 'BicycleM', 'BicycleT']].apply(pd.to_numeric)

CityJoint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27220 entries, 0 to 27219
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Area(sqKm)         27220 non-null  float64       
 1   BicycleF           27220 non-null  int64         
 2   BicycleM           27220 non-null  int64         
 3   BicycleT           27220 non-null  int64         
 4   BikeDensity        27220 non-null  float64       
 5   Bike_Colour        25102 non-null  object        
 6   Bike_Make          26458 non-null  object        
 7   Bike_Model         16801 non-null  object        
 8   Bike_Speed         27220 non-null  int64         
 9   Bike_Type          26949 non-null  object        
 10  City               27220 non-null  object        
 11  Cost_of_Bike       25052 non-null  float64       
 12  Division           27220 non-null  object        
 13  Dwellings          27220 non-null  int64         
 14  GeoUID

In [13]:
Data_Ottawa_pop=Data_Ottawa_pop.rename(columns={'Type_x':'Type', 'Region Name_x':'RegionName', 'Area (sq km)_x':'Area(sqKm)', 'Population _x':'Population',
       'Dwellings _x':'Dwellings', 'Households _x':'Households'})

Data_Toronto_pop=Data_Toronto_pop.rename(columns={'Type_x':'Type', 'Region Name_x':'RegionName', 'Area (sq km)_x':'Area(sqKm)', 'Population _x':'Population',
       'Dwellings _x':'Dwellings', 'Households _x':'Households'})

In [14]:
#concatening to just get the popluation and the bikes
Data_TOR=pd.concat([Data_Toronto_pop,Data_Ottawa_pop],ignore_index=True)
Data_TOR.drop(['v_CA16_1: Age Stats', 'Type_y', 'Region Name_y',
       'Area (sq km)_y', 'Population _y', 'Dwellings _y', 'Households _y'],axis=1,inplace=True)
#Data_TOR.shape , Data_Ottawa_pop.shape , Data_toronto_pop.shape
# Data_TOR.columns
# Data_TOR['GeoUID'].nunique()
# Data_TOR.info()



In [15]:
def drop_x(df):
    o_drop = [x for x in df if x.endswith('_x')]
    df.drop(o_drop, axis=1, inplace=True)

In [16]:
def ForDataFrameYear(Data_TOR,CityJoint,n_year):
    CityJoint_y=CityJoint[CityJoint['Occurrence_Year']==n_year]
    CityJoint_y = CityJoint_y.merge(Data_TOR,  how='right', on='GeoUID', suffixes=('_x', ''))
    drop_x(CityJoint_y)
    CityJoint_y["Status"].fillna("Bicycle_No_Theft", inplace = True) 
    CityJoint_y['City'].fillna(CityJoint_y['RegionName'],inplace=True)
    CityJoint_y['City'].replace({'Clarence-Rockland':'Ottawa','Russell':'Ottawa','North Grenville':'Ottawa'},inplace=True)
    return CityJoint_y

In [17]:
CityJoint_2014= ForDataFrameYear(Data_TOR,CityJoint,2014)
CityJoint_2015= ForDataFrameYear(Data_TOR,CityJoint,2015)
CityJoint_2016= ForDataFrameYear(Data_TOR,CityJoint,2016)
CityJoint_2017= ForDataFrameYear(Data_TOR,CityJoint,2017)
CityJoint_2018= ForDataFrameYear(Data_TOR,CityJoint,2018)
CityJoint_2019= ForDataFrameYear(Data_TOR,CityJoint,2019)

In [27]:
# Read data & geomap with respect to city
def read_data_map(data_path, geo_path, city = None):
    skip_cols = ['City']
    
    # Read data of Toronto an Ottawa cities
    canada_data = pd.read_csv(data_path, dtype = {"GeoUID":str})
    if "GeoUID" in canada_data.columns:
        canada_data["GeoUID"] = canada_data["GeoUID"].apply(lambda x: x if len(x.split(".")[-1]) > 1 else x + "0") # Fix important bug
    if city is not None:
        canada_data = canada_data[canada_data["City"] == city].drop(columns = skip_cols)
    
    if "%Theft/sqKm" in canada_data.columns:
        canada_data["%Theft/sqKm"] = canada_data["%Theft/sqKm"].replace({np.inf:0}) * 100
    if "%PopulationBike" in canada_data.columns:
        canada_data["%PopulationBike"] = canada_data["%PopulationBike"].replace({np.inf:0}) * 100
    
    # Read maps information
    canada_map = geopandas.read_file(geo_path, driver = "GeoJSON").rename(columns = {"id":"GeoUID"})
    canada_map["GeoUID"] = canada_map["GeoUID"].apply(lambda x: x if len(x.split(".")[-1]) > 1 else x + "0") # Fix important bug
    return canada_data, canada_map

# Update pandas to geopandas
def to_geo(df):
    try:
        df_point = geopandas.GeoDataFrame(df, geometry = geopandas.points_from_xy(df.x, df.y)).set_crs("EPSG:4326")
    except:
        df_point = geopandas.GeoDataFrame(df, geometry = geopandas.points_from_xy(df.Long, df.Lat)).set_crs("EPSG:4326")
    return df_point

# Join geopandas with data with two possibles cases (with column or with points in polygon)
def mapJoinTable(df_point, df_map, how = "right", keep_geo_cols = False, print_info = True):
    if "GeoUID" in df_point:
        df_point["GeoUID"] = df_point["GeoUID"].astype(str); df_map["GeoUID"] = df_map["GeoUID"].astype(str)
        how = "left" if how == "right" else "right" if how == "left" else how # Change how method
        df_result = df_map.join(df_point.set_index("GeoUID"), how = how, on = "GeoUID")#.rename(columns = {"id":"GeoUID"})
    else:
        df_point = to_geo(df_point)
        df_result = geopandas.tools.sjoin(df_point, df_map, how = how)
    if not keep_geo_cols: df_result = df_result.drop(columns = df_map.columns.drop(["GeoUID","geometry"]))
    if print_info:
        print("[INFO] Map, unique GeoUID and merge data lenght, respectively:",len(df_map), len(df_result["GeoUID"].unique()), len(df_result))
        print("[INFO] Total data not-merge successfully:", len(df_result[df_result["geometry"].isnull()]))        
    return df_result.reset_index(drop=True)

# Plot a single map
def plotMap(df_geomap, value_var, caption = "Cost of bike by region", zoom_start = 10,
            name = 'Canada maps', fields2show = None, aliases = None, index = True):
    df_data_geomap = df_geomap.copy().fillna(0) 
    
    # Map creation colors
    min_cn, max_cn = df_data_geomap[value_var].quantile([0.01,0.99]).apply(round, 2)
    index = df_data_geomap[value_var].quantile([0.2,0.4,0.6,0.8]) if index else None
    colormap = branca.colormap.LinearColormap(
        colors=['white','yellow','orange','red','darkred'], vmin=min_cn, vmax=max_cn, index = index
    )
    colormap.caption = caption
    
    # Geo map skeleton
    init_local = [z[0] for z in df_data_geomap.iloc[0]["geometry"].centroid.xy][::-1]
    geomap = folium.Map(location = init_local, zoom_start=zoom_start, tiles="OpenStreetMap")

    style_function = lambda x: {
        'fillColor': colormap(x['properties'][value_var]),
        'color': 'black', 'weight':2, 'fillOpacity':0.5
    }
    
    if fields2show is None: fields2show = list(df_data_geomap.columns.drop("geometry"))
    if aliases is None: aliases = fields2show.copy()
    stategeo = folium.GeoJson(df_data_geomap.to_json(), name=name, style_function = style_function,
        tooltip = folium.GeoJsonTooltip(fields = fields2show, aliases = aliases, localize=True)
    ).add_to(geomap)

    colormap.add_to(geomap)
    return geomap

In [29]:
### Toronto
save, year = False, None
df, df_geomap = read_data_map("../input/geo-dataset/Bicycle_Thefts_Toronto.csv", "../input/geo-dataset/pop_toronto.geojson")
if save:
    df_new2 = mapJoinTable(df, df_geomap, "left")
    print(len(df_new2), len(df_new2[df_new2["GeoUID"].isnull()]), len(df_new2[df_new2["GeoUID"].notnull()]))
    df_new2[df_new2["GeoUID"].notnull()].drop(columns = "geometry").to_csv("./Data/Bicycle_Thefts_Toronto_geo.csv", index = False)
df_new2 = mapJoinTable(df, df_geomap, "right")
# display(df_new2, df_new2.columns)

# Geomap proof 
status = list(df_new2["Status"].unique())
df_new2["new_status"] = df_new2["Status"].apply(lambda x: status.index(x) + 1 if x == x else 0)
if year is not None: df_new2 = df_new2[df_new2["Occurrence_Year"] == year]
toro = plotMap(df_new2.groupby("GeoUID").head(1), value_var = "new_status", 
            caption = "Status", name = 'Canada maps', index=False,fields2show = ["Status"])

toro

[INFO] Map, unique GeoUID and merge data lenght, respectively: 572 572 21589
[INFO] Total data not-merge successfully: 0


In [31]:
### Ottawa
df, df_geomap = read_data_map("../input/geo-dataset/Bicycle_Theft_Ottawa.csv", "../input/geo-dataset/pop_ottawa.geojson")
if save:
    df_new2 = mapJoinTable(df, df_geomap, "left")
    print(len(df_new2), len(df_new2[df_new2["GeoUID"].isnull()]), len(df_new2[df_new2["GeoUID"].notnull()]))
    df_new2[df_new2["GeoUID"].notnull()].drop(columns = "geometry").to_csv("./Data/Bicycle_Theft_Ottawa_geo.csv", index = False)
df_new2 = mapJoinTable(df, df_geomap, "right")
# display(df_new2, df_new2.columns)

# Geomap proof 
status = list(df_new2["Bicycle_Status"].unique())
df_new2["new_status"] = df_new2["Bicycle_Status"].apply(lambda x: status.index(x) + 1 if x == x else 0)
if year is not None: df_new2 = df_new2[df_new2["Report_Date"].str.contains(str(year))]
otaw = plotMap(df_new2.groupby("GeoUID").head(1), value_var = "new_status", 
            caption = "Bicycle_Status", name = 'Canada maps', index=False)

otaw

[INFO] Map, unique GeoUID and merge data lenght, respectively: 188 188 5652
[INFO] Total data not-merge successfully: 0


In [33]:
# Read geojson/data and put in pandas dataframe
toronto_data, toronto_map = read_data_map(data_path = "../input/miscdata/Stolen_bikes_johan_2016.csv", 
                                        geo_path = "../input/geo-dataset/pop_toronto.geojson", 
                                        city = "Toronto")

toronto_data_geomap = mapJoinTable(toronto_data, toronto_map, how = "right")
# display(toronto_data_geomap.head().append(toronto_data_geomap.tail()), toronto_data_geomap.shape, toronto_data_geomap.columns)

# For demo
max_hour = toronto_data_geomap.groupby("Hour").size().idxmax()
toronto_data_demo = toronto_data_geomap[toronto_data_geomap["Hour"] == max_hour].copy() # Important data = Demo
display(toronto_data_demo.head().append(toronto_data_demo.tail()), toronto_data_demo.shape, toronto_data_demo.columns)

[INFO] Map, unique GeoUID and merge data lenght, respectively: 572 572 2209
[INFO] Total data not-merge successfully: 0


,GeoUID,geometry,Unnamed: 0,Hour,PopulationCT,BicycleT,Area(sqKm),SumThBikes,BikesThDensity,BikesDensity,PopulationDensity,%Theft/sqKm,%PopulationBike
16,5350164.00,"MULTIPOLYGON (((-79.43183 43.68881, -79.43125 ...",2181.0,12.0,6258.0,195.0,0.62823,1.0,1.591774,310.395874,9961.319899,0.512821,3.116012
49,5350071.00,"MULTIPOLYGON (((-79.35029 43.67130, -79.35278 ...",1694.0,12.0,3481.0,185.0,0.46398,4.0,8.621061,398.724083,7502.478555,2.162162,5.314565
64,5350010.02,"MULTIPOLYGON (((-79.41122 43.64012, -79.41318 ...",842.0,12.0,7740.0,370.0,0.54735,4.0,7.307938,675.984288,14140.860510,1.081081,4.780362
75,5350010.01,"MULTIPOLYGON (((-79.41122 43.64012, -79.41459 ...",823.0,12.0,5428.0,225.0,0.26263,2.0,7.615276,856.718577,20667.859727,0.888889,4.145173
90,5350075.00,"MULTIPOLYGON (((-79.32966 43.67494, -79.33095 ...",1730.0,12.0,4083.0,125.0,0.49858,2.0,4.011392,250.712022,8189.257491,1.600000,3.061474
2135,5350073.00,"MULTIPOLYGON (((-79.34227 43.66775, -79.34188 ...",1712.0,12.0,2605.0,80.0,0.35718,1.0,2.799709,223.976706,7293.241503,1.250000,3.071017
2143,5350074.00,"MULTIPOLYGON (((-79.32801 43.67092, -79.32966 ...",1720.0,12.0,3848.0,205.0,0.39980,1.0,2.501251,512.756378,9624.812406,0.487805,5.327443
2162,5350052.00,"MULTIPOLYGON (((-79.44384 43.65086, -79.44532 ...",1430.0,12.0,4077.0,295.0,0.50129,1.0,1.994853,588.481717,8133.016817,0.338983,7.235713
2182,5350196.02,"MULTIPOLYGON (((-79.36929 43.71167, -79.36892 ...",2289.0,12.0,5635.0,75.0,1.10548,1.0,0.904584,67.843833,5097.333285,1.333333,1.330967
2204,5350063.06,"MULTIPOLYGON (((-79.38309 43.66137, -79.38504 ...",1610.0,12.0,5171.0,130.0,0.19221,4.0,20.810572,676.343583,26902.866656,3.076923,2.514020


(128, 13)

Index(['GeoUID', 'geometry', 'Unnamed: 0', 'Hour', 'PopulationCT', 'BicycleT',
       'Area(sqKm)', 'SumThBikes', 'BikesThDensity', 'BikesDensity',
       'PopulationDensity', '%Theft/sqKm', '%PopulationBike'],
      dtype='object')

In [34]:
print("[INFO]: Hour =", max_hour)
fields = toronto_data_demo.columns.drop(['GeoUID', 'Hour', 'Unnamed: 0', 'geometry']).to_list()
plotMap(toronto_data_demo, value_var = "%Theft/sqKm", caption = "%Theft/sqKm of bike by region", 
        name = 'Canada maps', index = False, fields2show = fields)

[INFO]: Hour = 12.0
